In [ ]:
!pip -q install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 KB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
from google.colab import drive
import imageio as iio
import numpy as np
import timeit

In [ ]:
drive.mount('/content/drive')
dir_images ='/content/drive/MyDrive/Multithreaded_and_Distributed_Programming/LB6B/pic'

Mounted at /content/drive


# Checking Cuda

In [ ]:
 %%sh 
cat > hello.cu << EOF
#include <stdio.h>

int main() {
  int devices;
  cudaGetDeviceCount(&devices);

  for (int d = 0; d < devices; ++d) {
    cudaDeviceProp p;
    cudaGetDeviceProperties(&p, d);
    int mp = p.multiProcessorCount, sp = 0;
    if (p.major == 2) {
      if (p.minor == 1) sp = 48;
      else sp = 32;
    } else if (p.major == 3) {
      sp = 192;
    } else if (p.major == 5) {
      sp = 128;
    }
    printf("Device %d: %s\n", d, p.name);
    printf(" -> multiprocessor count: %d\n", mp);
    printf(" -> stream processor count: %d (total %d)\n", sp, sp * mp);
    printf(" -> warp size: %d\n", p.warpSize);
    printf(" -> max threads per block: %d\n", p.maxThreadsPerBlock);
    printf(" -> max block dimensions: %d x %d x %d\n", p.maxThreadsDim[0], p.maxThreadsDim[1], p.maxThreadsDim[2]);
    printf(" -> max grid dimensions: %d x %d x %d\n", p.maxGridSize[0], p.maxGridSize[1], p.maxGridSize[2]);
    puts("");
  }
  return 0;
}
EOF

In [ ]:
 !nvcc -o hello_cuda hello.cu && ./hello_cuda

Device 0: Tesla T4
 -> multiprocessor count: 40
 -> stream processor count: 0 (total 0)
 -> warp size: 32
 -> max threads per block: 1024
 -> max block dimensions: 1024 x 1024 x 64
 -> max grid dimensions: 2147483647 x 65535 x 65535



# CPU

In [ ]:
# 1024 x 768
pic1 = iio.imread(f"{dir_images}/1.jpg")
pic2 = iio.imread(f"{dir_images}/1.1.jpg")

# 1280 x 960
# pic1 = iio.imread(f"{dir_images}/2.jpg")
# pic2 = iio.imread(f"{dir_images}/2.1.jpg")

# 2048 x 1536
# pic1 = iio.imread(f"{dir_images}/3.jpg")
# pic2 = iio.imread(f"{dir_images}/3.1.jpg")

rows = pic1.shape[0]
cols = pic1.shape[1]

start_timer =timeit.default_timer()

pic1r =  np.array([[(int(pic1[i,j, 0])) for j in range(cols)] for i in range(rows)])
pic1g =  np.array([[(int(pic1[i,j, 1])) for j in range(cols)] for i in range(rows)])
pic1b =  np.array([[(int(pic1[i,j, 2])) for j in range(cols)] for i in range(rows)])
pic2r =  np.array([[(int(pic2[i,j, 0])) for j in range(cols)] for i in range(rows)])
pic2g =  np.array([[(int(pic2[i,j, 1])) for j in range(cols)] for i in range(rows)])
pic2b =  np.array([[(int(pic2[i,j, 2])) for j in range(cols)] for i in range(rows)])
arr1 = np.around(((pic1r + pic1g + pic1b) / 3)).astype("int16")
arr2 = np.around(((pic2r + pic2g + pic2b) / 3)).astype("int16")
arr3 = arr1 - arr2
arr4 = np.around((arr3 + 255)/ 2).astype("uint8")
filename = dir_images+"/result_cuda_сpu_" + str(arr4.shape[1]) +"_"+ str(arr4.shape[0]) +".jpeg"
result = iio.imwrite(filename, arr4, extension=".jpeg")

time = (timeit.default_timer()-start_timer)
print('Время вычисления на CPU: {:2.4f}'.format(time))

<ipython-input-7-598689a47b96>:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic1 = iio.imread(f"{dir_images}/1.jpg")
<ipython-input-7-598689a47b96>:3: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic2 = iio.imread(f"{dir_images}/1.1.jpg")


Время вычисления на CPU: 2.3737


# GPU Cuda

In [ ]:
#!python 
import pycuda.driver as drv
import pycuda.autoinit
from pycuda.compiler import SourceModule

mod = SourceModule \
    (
        """
#include<stdio.h>
#define INDEX1(a1, b1) a1*1024+b1
#define INDEX2(a2, b2) a2*1024+b2
#define INDEX3(a3, b3) a3*1024+b3
#define INDEX4(a4, b4) a4*1024+b4

__global__ void subtract_images(float *dest_pic1, float *dest_pic2, float *dest_pic3, float *dest_pic4, float *pic1_r_img, float *pic1_g_img, float *pic1_b_img, float *pic2_r_img, float *pic2_g_img, float *pic2_b_img )
{

unsigned int idx = threadIdx.x+(blockIdx.x*(blockDim.x*blockDim.y));

  unsigned int a1 = idx/1024;
  unsigned int b1 = idx%1024;
  unsigned int a2 = idx/1024;
  unsigned int b2 = idx%1024;
  unsigned int a3 = idx/1024;
  unsigned int b3 = idx%1024;
  unsigned int a4 = idx/1024;
  unsigned int b4 = idx%1024;

dest_pic1[INDEX1(a1, b1)] = ((pic1_r_img[INDEX1(a1, b1)]/3)+(pic1_g_img[INDEX1(a1, b1)]/3)+(pic1_b_img[INDEX1(a1, b1)]/3));
dest_pic2[INDEX2(a2, b2)] = ((pic2_r_img[INDEX2(a2, b2)]/3)+(pic2_g_img[INDEX2(a2, b2)]/3)+(pic2_b_img[INDEX2(a2, b2)]/3));
dest_pic3[INDEX3(a3, b3)] = dest_pic1[INDEX1(a1, b1)] - dest_pic2[INDEX2(a2, b2)];
dest_pic4[INDEX4(a4, b4)] = ((dest_pic3[INDEX3(a3, b3)]+255)/2);
}

"""
    )

# 1024 x 768
pic1 = iio.imread(f"{dir_images}/1.jpg").astype(np.float32)
pic2 = iio.imread(f"{dir_images}/1.1.jpg").astype(np.float32)

# 1280 x 960
# pic1 = iio.imread(f"{dir_images}/2.jpg").astype(np.float32)
# pic2 = iio.imread(f"{dir_images}/2.1.jpg").astype(np.float32)

# 2048 x 1536
# pic1 = iio.imread(f"{dir_images}/3.jpg").astype(np.float32)
# pic2 = iio.imread(f"{dir_images}/3.1.jpg").astype(np.float32)

rows = pic1.shape[0]
cols = pic1.shape[1]
size= rows*cols
start_timer =timeit.default_timer()
pic1_r_img = pic1[:, :, 0].reshape(size, order='F')
pic1_g_img = pic1[:, :, 1].reshape(size, order='F')
pic1_b_img = pic1[:, :, 2].reshape(size, order='F')
pic2_r_img = pic2[:, :, 0].reshape(size, order='F')
pic2_g_img = pic2[:, :, 1].reshape(size, order='F')
pic2_b_img = pic2[:, :, 2].reshape(size, order='F')
dest_pic1 = np.zeros_like(pic1_r_img)
dest_pic2 = np.zeros_like(pic2_r_img)
dest_pic3 = np.zeros_like(pic1_r_img)
dest = np.zeros_like(pic1_r_img)
subtract_images = mod.get_function("subtract_images")
subtract_images(drv.Out(dest_pic1), drv.Out(dest_pic2), drv.Out(dest_pic3), drv.Out(dest), drv.In(pic1_r_img), drv.In(pic1_g_img), drv.In(pic1_b_img), drv.In(pic2_r_img), drv.In(pic2_g_img), drv.In(pic2_b_img), block=(1024, 1, 1), grid=(1024, 1, 1))

dest=np.reshape(dest,(rows, cols), order='F')
dest = dest.astype(np.uint8)

filename = dir_images+"/result_cuda_gpu_" + str(dest.shape[1]) +"_"+ str(dest.shape[0]) +".jpeg"
result = iio.imwrite(filename, dest, extension=".jpeg")

time = (timeit.default_timer()-start_timer)
print('Время вычисления на GPU Cuda: {:2.4f}'.format(time))

<ipython-input-8-66896eba011c>:39: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic1 = iio.imread(f"{dir_images}/1.jpg").astype(np.float32)
<ipython-input-8-66896eba011c>:40: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  pic2 = iio.imread(f"{dir_images}/1.1.jpg").astype(np.float32)


Время вычисления на GPU Cuda: 0.5738
